In [1]:
import pandas as pd
import xlrd
import arcpy
from pandas import ExcelWriter
from datetime import datetime
import os

In [2]:
now = datetime.now().strftime('%Y%m%d%H%M%S')
temp_path = r'C:\temp\data_processing_exports'
questionnaire_file = r'C:\git\hh_survey\R3 questionnaire_GeoPoll_final.xlsx' #the questionnaire file that we use for creating the table
coded_values_file = os.path.join(temp_path, "coded_values_%s.xlsx" % now) #intermediary output file with all categories and codes extracted from the questionnaire
writer = pd.ExcelWriter(coded_values_file, engine='xlsxwriter')
field_names_list = []
max_counter = 3000 #for testing purposes, we may need to limit the execution only to some items

In [3]:
def importallsheets(in_excel, out_gdb):
    ###this function import all sheets of an xlsx file into separate tables inside a GDB (with same name)
    print("max_counter: %s" % max_counter)
    counter = 0
    workbook = xlrd.open_workbook(in_excel)
    sheets = [sheet.name for sheet in workbook.sheets()]

    print('{} sheets found: {}'.format(len(sheets), ','.join(sheets)))
    for sheet in sheets:
        counter +=1
        if counter <= max_counter:
            # The out_table is based on the input excel file name
            # a underscore (_) separator followed by the sheet name
            out_table = os.path.join(
                out_gdb,
                arcpy.ValidateTableName(
                    "{0}".format(sheet),
                    out_gdb))

            print('Converting {} to {}'.format(sheet, out_table))

            # Perform the conversion
            arcpy.ExcelToTable_conversion(in_excel, out_table, sheet)
            

def make_attribute_dict(fc, code_field, value_field):
    #this function creates a dictionary based on a GDB table
    attdict = {}
    with arcpy.da.SearchCursor(fc, [code_field, value_field]) as cursor:
        for row in cursor:
            attdict[row[0]] = row[1]
    return attdict

def fix_category_formatting(category):
    #this function improves and standardizes the formatting of the categories' descriptions
     return category.replace("[","(").replace("]",")").replace("(specify)","").replace("/ ",", ").capitalize().replace("adps","ADPs").replace("idp","IDP").replace("covid","COVID")


In [4]:
##this section of the script reads the survey excel file and creates an excel file with multiple sheets:
### each sheet contains the coded value and description for a "Single choice" or "Open Ended-Select All That Apply" question.
###moreover, it creates and populates several lists that will be used later for defining each field of the final table names, types and domains

print("Opening questionnaire DF")
quest_df = pd.read_excel(open(questionnaire_file, 'rb'), sheet_name='Questionnaire HH',skiprows=2)
#create a list of all possible numbering
numbering = ["%s)" % n for n in range(1,200)] ## 1), 2), ... 200)
# initialize list of lists that will store the results
dict_derived_fieldnames = {} #this dict will group all derived fields in case of "Select All That Apply" type of questions
field_names_list = [] ##this list will contain all fields of the final table
text_type_fields = [] ##this list will contain all fields of the final table with TEXT type
range_type_fields = [] ##this list will contain all fields of the final table storing RANGE data  (will be LONG type)
##iterate the following for each row (so each question of the questionnaire)
for index, row in quest_df.iterrows():
    try:
        first_derived_fieldname = "" #the name of the first derived field will be the main of the domain table
        all_derived_fieldnames = []
        codes_and_labels = []
        categories = str(row['English']).replace("\t","")
        question_name = row['Suggested Qname']  #Q Name
        question_type = row['Q Type']
        programming_instructions = row['Programming Instructions'] #this field contains coded values for crop_main
        print("\n\n----%s----" % question_name)
        #only for questions with pre-defined categories need domains
        if question_type in ("StartRecording","Single Choice","Open Ended-Single Choice", "Open Ended - Single Choice", "Open Ended-Select All That Apply",
                             "Select All That Apply","Open Ended - Select All That Apply "):
            if question_name == 'crp_main': #for this question only, coded values should be taken from field programming_instructions
                programming_lines = programming_instructions.splitlines()
                for programming_line in programming_lines:
                    if ")" in programming_line:
                        #print(programming_line)
                        index, category = programming_line.split(")")
                        category = fix_category_formatting(category)
                        codes_and_labels.append([index, category])
            else:
                #find all numbering present in the category string
                numbering_in_text = [n for n in numbering if n in categories]
                #print(numbering_in_text)
                ##the following loop creates a list "codes_and_labels" with all available codes&labels for each question
                for index in range(0,len(numbering_in_text)):
                    start = categories.find(numbering_in_text[index]) + len(numbering_in_text[index])
                    try:
                        end = categories.find(numbering_in_text[index + 1])
                        substring = categories[start:end].strip()
                    except:
                        # it fails during the last loop -> the last option is usually at the end of the string
                        substring = categories[start:].strip()
                    #print(substring)
                    category = fix_category_formatting(substring)
                    codes_and_labels.append([index +1, category])
                
            if question_type not in ["Open Ended-Select All That Apply","Select All That Apply","Open Ended - Select All That Apply "]:
                #so questions with NO derived fields
                field_names_list.append(question_name)
                codes_and_labels_df = pd.DataFrame(codes_and_labels, columns=['code', 'label'])
                codes_and_labels_df.to_excel(writer, sheet_name=question_name)
            else:
                #so questions with derived fields
                numbering_in_qname = [n for n in numbering if n in question_name]
                for index in range(0, len(numbering_in_qname)):
                    start = question_name.find(numbering_in_qname[index]) + len(numbering_in_qname[index])
                    try:
                        end = question_name.find(numbering_in_qname[index + 1])
                        derived_field_name = question_name[start:end].strip()
                    except:
                        # it fails during the last loop -> the last option is usually at the end of the string
                        derived_field_name = question_name[start:].strip()
                    if index == 0:
                        first_derived_fieldname = derived_field_name
                        codes_and_labels_df = pd.DataFrame(codes_and_labels, columns=['code', 'label'])
                        codes_and_labels_df.to_excel(writer, sheet_name=derived_field_name)
                    all_derived_fieldnames.append(derived_field_name)
                    field_names_list.append(derived_field_name)
                dict_derived_fieldnames[first_derived_fieldname] = all_derived_fieldnames
        elif question_type == "Range":
            #these questions will be associated to LONG type fields
            field_names_list.append(question_name)
            range_type_fields.append(question_name)
        elif question_type == "Open Ended":
            #these questions will be associated to TEXT type fields
            if not pd.isnull(question_name): #NaN rows we want to skip (i.e. OptIn question without a name in the survey)
                field_names_list.append(question_name)
                text_type_fields.append(question_name)
        else:
            print("QUESTION SKIPPED ---------", question_name, question_type)
    except:
        print("failed for some reasons")


print("Saving codes and labels %s" % coded_values_file)
# Close the Pandas Excel writer and output the Excel file.
writer.save()

Opening questionnaire DF


----nan----
QUESTION SKIPPED --------- nan nan


----nan----


----calldispo----


----calldispo_answeringmachine----
QUESTION SKIPPED --------- calldispo_answeringmachine nan


----calldispo_noanswer----
QUESTION SKIPPED --------- calldispo_noanswer nan


----calldispo_underreview----
QUESTION SKIPPED --------- calldispo_underreview nan


----calldispo_disconnected----
QUESTION SKIPPED --------- calldispo_disconnected nan


----resp_language----


----introduction----


----resp_agree----


----resp_refusalwhy----


----resp_whencallback----


----callbackmessage_en----


----nan----
QUESTION SKIPPED --------- nan nan


----NA ----
QUESTION SKIPPED --------- NA  nan


----nan----
QUESTION SKIPPED --------- nan nan


----resp_age----


----resp_gender----


----hh_admin1----


----hh_admin2_1----


----hh_admin2_2----


----quotareached----


----hh_agricactivity----


----nan----
QUESTION SKIPPED --------- nan nan


----hh_gender----


----hh_education----



25) need_ref----


----need_received_

1) need_received_food
2) need_received_cashvouchers
3) need_received_seeds
4) need_received_extensionservices
5) need_received_livestockfeed
6) need_received_other
7) need_received_noassistreceived
8) need_received_dk
9) need_received_ref----


----nan----
QUESTION SKIPPED --------- nan nan


----callback----


----language2----


----closeout----
QUESTION SKIPPED --------- closeout nan
Saving codes and labels C:\temp\data_processing_exports\coded_values_20210504151635.xlsx


In [5]:
###this section creates a new GDB
gdb_name = "fGDB_with_coded_values_%s.gdb" % now
output_gdb = os.path.join(temp_path, gdb_name)
arcpy.CreateFileGDB_management(temp_path,gdb_name)

<Result 'C:\\temp\\data_processing_exports\\fGDB_with_coded_values_20210504151635.gdb'>

In [6]:
##here we and import each table with coded values and description into the GDB
###it can take up to 5 seconds per table - and we normally process hundreds of table.
####for this reason, for testing purposes edit variable max_counter in the first cell of this notebook


importallsheets(coded_values_file, output_gdb)

max_counter: 3000
95 sheets found: calldispo,resp_language,introduction,resp_agree,resp_refusalwhy,resp_gender,hh_admin1,hh_admin2_1,hh_admin2_2,quotareached,hh_agricactivity,hh_gender,hh_education,hh_wealth,hh_residencetype,hh_maritalstat,income_main,income_main_amount_conf,income_main_comp,income_sec,income_sec_amount_conf,income_sec_comp,income_third,income_third_amount_conf,income_third_comp,covid_goodstransp,shock_noshock,crp_main,crp_landsize,crp_landright,crp_irrigation,crp_seed_ownprod,crp_area_change,crp_harv_change,crp_proddif,crp_proddif_plantdisease,crp_salesdif,crp_saledif_marketing_cost,crp_salesprice,crp_proc,crp_proc_mach_mill,crp_proc_owner_yours,crp_proc_state,ls_main,ls_num_nochange,ls_food_supply_openpasture,ls_proddif,ls_proddif_diffeedpurchase,ls_salesmain,ls_salesdif,ls_salesdif_smallerprofits,ls_salesprice,ls_proc,ls_proc_slaughter,ls_proc_fac_priv_cold_transp,ls_proc_fac_state,fish_main_coastal,fish_change,fish_proddif,fish_proddif_find,fish_inputdif_bait,fish_

Converting fish_main_coastal to C:\temp\data_processing_exports\fGDB_with_coded_values_20210504151635.gdb\fish_main_coastal
Converting fish_change to C:\temp\data_processing_exports\fGDB_with_coded_values_20210504151635.gdb\fish_change
Converting fish_proddif to C:\temp\data_processing_exports\fGDB_with_coded_values_20210504151635.gdb\fish_proddif
Converting fish_proddif_find to C:\temp\data_processing_exports\fGDB_with_coded_values_20210504151635.gdb\fish_proddif_find
Converting fish_inputdif_bait to C:\temp\data_processing_exports\fGDB_with_coded_values_20210504151635.gdb\fish_inputdif_bait
Converting fish_salesmain to C:\temp\data_processing_exports\fGDB_with_coded_values_20210504151635.gdb\fish_salesmain
Converting fish_salesdif_1 to C:\temp\data_processing_exports\fGDB_with_coded_values_20210504151635.gdb\fish_salesdif_1
Converting fish_saledif_smallerprofits to C:\temp\data_processing_exports\fGDB_with_coded_values_20210504151635.gdb\fish_saledif_smallerprofits
Converting fish_sa

In [9]:
###it reads the survey file and creates an excel file with all field names of the outputs table.
###then it imports the empty output table template to the GDB, and calls it survey_data. this table will be the future master table
###coded values will be enforced there.

#insert opening fields that come from Geopoll and are not captured by the questionnaire file
opening_fields = [["survey_id",'TEXT'],["operator_id",'TEXT'],["adm0_name",'TEXT'],["adm0_ISO3",'TEXT'],["adm1_pcode",'TEXT'], ["adm1_name",'TEXT'],["adm2_pcode",'TEXT'],["adm2_name",'TEXT'],
["survey_created_date",'TEXT'],["opt_in_date",'TEXT'],["total_case_duration",'TEXT']]

#manage opening fields type
opening_fields_names_text = [i[0] for i in opening_fields if i[1] == 'TEXT']
opening_fields_names_short = [i[0] for i in opening_fields if i[1] == 'SHORT']
opening_fields_names_range = [i[0] for i in opening_fields if i[1] == 'RANGE']
opening_fields_names_date = [i[0] for i in opening_fields if i[1] == 'DATE']

text_type_fields += opening_fields_names_text
range_type_fields += opening_fields_names_range

opening_field_names = [i[0] for i in opening_fields]
field_names_list = opening_field_names + field_names_list

#some fields in the questionnaire should be removed from the master table
useless_fields = ['callbackmessage_en', 'hh_admin1', 'calldispo', 'hh_admin2_1', 'hh_admin2_2', 'quotareached', 'resp_refusalwhy', 'resp_whencallback', 'fies','copingstrategies',
                 "introduction",'hdds_confirmation','income_main_amount_conf','income_sec_amount_conf','income_third_amount_conf']


s_useless_fields = set(useless_fields)  

field_names_list = [x for x in field_names_list if x not in s_useless_fields]


survey_empty_table_df = pd.DataFrame(columns=[field_names_list])
survey_empty_table_xlsx = os.path.join(temp_path, "survey_empty_table_%s.xlsx" % now)
writer = pd.ExcelWriter(survey_empty_table_xlsx, engine='xlsxwriter')
survey_empty_table_df.to_excel(writer, sheet_name="survey_data")
writer.save()

In [10]:
###this section imports the excel file just created into the GDB

importallsheets(survey_empty_table_xlsx, output_gdb)


max_counter: 3000
1 sheets found: survey_data
Converting survey_data to C:\temp\data_processing_exports\fGDB_with_coded_values_20210504151635.gdb\survey_data


In [11]:
##there is always an unwanted field to remove
try:
    arcpy.DeleteField_management(os.path.join(output_gdb, "survey_data"),"COL_A")
except:
    print ("field COL_A does not exist")

In [12]:
### This section alters each fields type in the survey master table
###domain related fiels shuld be integer, in order to enforce coded values, except crop_main

##string fields: open ended questions + crop_main
##long fields: range and numbers related questions (age, currency, hh size...)
##short fields: coded values questions

GDB_survey_table = os.path.join(output_gdb,"survey_data")
field_names = [f.name for f in arcpy.ListFields(GDB_survey_table)]
for field in field_names:
    if field in field_names_list:
        print("changing type for field %s" % field)
        try:
            if field == 'crp_main' or field in text_type_fields: #crp_main is associated to text codes since they have possible dots (.) for subcategories
                arcpy.management.AlterField(GDB_survey_table, field, field_type = "TEXT")
            elif field in range_type_fields: #i.e. currencies
                arcpy.management.AlterField(GDB_survey_table, field, field_type = "LONG")
            else: #all other fields are short integer (simple  code values)
                arcpy.management.AlterField(GDB_survey_table, field, field_type = "SHORT")
        except:
            pass

changing type for field survey_id
changing type for field operator_id
changing type for field adm0_name
changing type for field adm0_ISO3
changing type for field adm1_pcode
changing type for field adm1_name
changing type for field adm2_pcode
changing type for field adm2_name
changing type for field survey_created_date
changing type for field opt_in_date
changing type for field total_case_duration
changing type for field resp_language
changing type for field resp_agree
changing type for field resp_age
changing type for field resp_gender
changing type for field hh_agricactivity
changing type for field hh_gender
changing type for field hh_education
changing type for field hh_wealth
changing type for field hh_residencetype
changing type for field hh_size
changing type for field hh_maritalstat
changing type for field income_main
changing type for field income_main_amount
changing type for field income_main_comp
changing type for field income_sec
changing type for field income_sec_amount
cha

changing type for field ls_proc_fac_coop_dairy
changing type for field ls_proc_fac_other
changing type for field ls_proc_fac_none
changing type for field ls_proc_fac_dk
changing type for field ls_proc_fac_ref
changing type for field ls_proc_fac_state
changing type for field fish_main_coastal
changing type for field fish_main_opensea
changing type for field fish_main_lakespondsrivers
changing type for field fish_main_aquaculture
changing type for field fish_main_dk
changing type for field fish_main_ref
changing type for field fish_change
changing type for field fish_proddif
changing type for field fish_proddif_find
changing type for field fish_proddif_covid
changing type for field fish_proddif_fuel
changing type for field fish_proddif_inputs
changing type for field fish_proddif_labour
changing type for field fish_proddif_other
changing type for field fish_proddif_dk
changing type for field fish_proddif_ref
changing type for field fish_inputdif_bait
changing type for field fish_inputdif_

In [13]:
def create_and_apply_domain(master_table, domain_table, field_name, domain_already_exists):
    # Process: Create the coded value domain
    domName = '%s_dom' % domain_table
    if not domain_already_exists:
        if domain_table == 'crp_main':
            field_type = "TEXT"
        else:
            field_type = "SHORT"
        try:
            arcpy.CreateDomain_management(output_gdb, domName, field_type= field_type)
        except:
            print('domain exists')
        # Store all the domain values in a dictionary with the domain code as the "key" 
        # and the domain description as the "value" (domDict[code])

        domDict = make_attribute_dict(domain_table, 'code', 'label')
        print(domDict)

        #   dict example:  {"CI":"Cast iron", "DI": "Ductile iron", "PVC": "PVC", 
        #                "ACP": "Asbestos concrete", "COP": "Copper"}

        # Process: Add valid material types to the domain
        # use a for loop to cycle through all the domain codes in the dictionary
        for code in domDict:
            try:
                arcpy.AddCodedValueToDomain_management(output_gdb, domName, code, domDict[code])
            except:
                print("AddCodedValueToDomain_management failed")

        # Process: Constrain the material value of distribution mains

    arcpy.AssignDomainToField_management(master_table, field_name, domName) 

In [14]:
##THis section of the script searches for all domains tables in the GDB
##then, for each table  it creates a new domain,
###it stores all domain values in a dictionary, it add each code domain, and finally assign the domain to the right field (having same name)

# Set the current workspace
arcpy.env.workspace = output_gdb

# Get and print a list of tables
tables = arcpy.ListTables()
master_table = os.path.join(output_gdb,'survey_data')
for table in tables:
    if 'survey_data' not in table and table not in useless_fields:
        #print(table)
        print ("Creating and Applying (new) domain for %s" % table)
        create_and_apply_domain(master_table, table, table, False) #domain_table has the same name of the field where it should be applied
        if table in dict_derived_fieldnames:
            derived_fields_list = dict_derived_fieldnames[table]
            for derived_field in derived_fields_list:
                print ("Creating and Applying (existing) domain for derivate field %s" % derived_field)
                create_and_apply_domain(master_table, table, derived_field, True)


        

Creating and Applying (new) domain for resp_language
{1: 'English', 2: 'Other options (add as many as necessary)'}
Creating and Applying (new) domain for resp_agree
{1: 'Yes', 2: 'Not now but another time in the week', 3: 'No'}
Creating and Applying (new) domain for resp_gender
{1: 'Male', 2: 'Female', 3: "Don't know", 4: 'Refused'}
Creating and Applying (new) domain for hh_agricactivity
{1: 'Yes - crop production', 2: 'Yes - livestock production', 3: 'Yes - both crop and livestock production', 4: 'No', 5: "Don't know", 6: 'Refused'}
Creating and Applying (new) domain for hh_gender
{1: 'Male', 2: 'Female', 3: "Don't know", 4: 'Refused'}
Creating and Applying (new) domain for hh_education
{1: 'None or did not complete primary school', 2: 'Completed primary school', 3: 'Completed secondary school', 4: 'Completed higher education (university , college) degree', 5: 'Islamic education', 6: "Don't know", 7: 'Refused'}
Creating and Applying (new) domain for hh_wealth
{}
Creating and Applying 

{1: 'No shock', 2: 'Sickness or death of household member(s) (sickness of breadwinner , unusually high medical expenditure , funeral expenses , death of breadwinner)', 3: 'Lost employment or working opportunities', 4: 'Other intra household shock', 5: 'Much higher than usual food prices', 6: 'Much higher than usual fuel prices', 7: 'External event impeding the continuation of work or business affecting all - laws closing markets , businesses , road infrastructure to markets destroyed, etc.', 8: 'Other economic shock', 9: 'Pest outbreak', 10: 'Plant disease', 11: 'Animal disease affecting many animals', 12: 'Lack of physical access to pasture (because of conflict , infrastructure damage , COVID-19 restrictions)', 13: 'Other crop and livestock shock', 14: 'Cold temperatures or hail', 15: 'Flood', 16: 'Hurricane , cyclone', 17: 'Drought', 18: 'Earthquake', 19: 'Landslides', 20: 'Fire from natural disaster', 21: 'Other natural hazard', 22: 'Violence and insecurity , conflict', 23: 'Theft o

Creating and Applying (existing) domain for derivate field crp_proddif_dk
Creating and Applying (existing) domain for derivate field crp_proddif_ref
Creating and Applying (new) domain for crp_salesdif
{1: 'Yes', 2: 'No', 3: "Don't know", 4: 'Refused'}
Creating and Applying (new) domain for crp_saledif_marketing_cost
{1: 'Higher marketing costs (such as transportation)', 2: 'Damage and losses due to delay or inability to physically access markets', 3: 'Usual traders or local customers are not buying as much as usual', 4: 'Prices are too low', 5: 'Difficulty processing product (lack of processing inputs, equipment, etc)', 6: 'Other ', 7: "Don't know", 8: 'Refused'}
Creating and Applying (existing) domain for derivate field crp_saledif_marketing_cost
Creating and Applying (existing) domain for derivate field crp_saledif_damagelosses
Creating and Applying (existing) domain for derivate field crp_saledif_lowdemand
Creating and Applying (existing) domain for derivate field crp_saledif_lowpri

Creating and Applying (existing) domain for derivate field ls_proddif_ref
Creating and Applying (new) domain for ls_salesmain
{1: 'Cattle (cow , beef , veal , yak , buffalo)', 2: 'Goats', 3: 'Sheeps', 4: 'Swine', 5: 'Equine (donkey , horse , etc.)', 6: 'Small domesticated mammals (rabbits , minks , guinea pigs , etc)', 7: 'Poultry (chicken , guineafowl , duck , etc.)', 8: 'Camelids', 9: 'Eggs', 10: 'Milk or dairy products', 11: 'Skin or leather', 12: "Don't know", 13: 'Refused'}
Creating and Applying (new) domain for ls_salesdif
{1: 'Yes', 2: 'No', 3: "Don't know", 4: 'Refused'}
Creating and Applying (new) domain for ls_salesdif_smallerprofits
{1: 'Higher marketing costs (such as transportation)', 2: 'Damage and losses due to delay or inability to physically access markets (including limited storage capacity)', 3: 'Usual traders or local customers are not buying as much as usual', 4: 'Prices are too low', 5: 'Closure of slaughterhouses or difficulties accessing slaughterhouses', 6: 'Di

Creating and Applying (existing) domain for derivate field fish_saledif_lowdemand
Creating and Applying (existing) domain for derivate field fish_saledif_lowprices
Creating and Applying (existing) domain for derivate field fish_saledif_processing
Creating and Applying (existing) domain for derivate field fish_saledif_other
Creating and Applying (existing) domain for derivate field fish_saledif_dk
Creating and Applying (existing) domain for derivate field fish_saledif_ref
Creating and Applying (new) domain for fish_salesprice
{1: 'A lot more (increased by more than 50%)', 2: 'More', 3: 'Same', 4: 'Less', 5: 'A lot less (less than half as much)', 6: "Don't know", 7: 'Refused'}
Creating and Applying (new) domain for fies_worried
{1: 'Yes', 2: 'No', 3: "Don't know", 4: 'Refused'}
Creating and Applying (new) domain for fies_healthy
{1: 'Yes', 2: 'No', 3: "Don't know", 4: 'Refused'}
Creating and Applying (new) domain for fies_fewfoods
{1: 'Yes', 2: 'No', 3: "Don't know", 4: 'Refused'}
Creati

Creating and Applying (existing) domain for derivate field need_accesstoland
Creating and Applying (existing) domain for derivate field need_animalfeed
Creating and Applying (existing) domain for derivate field need_veterinaryservices
Creating and Applying (existing) domain for derivate field need_veterinaryinputs
Creating and Applying (existing) domain for derivate field need_animalsalemingarantdprice
Creating and Applying (existing) domain for derivate field need_restockinganimals
Creating and Applying (existing) domain for derivate field need_supportforprocessprod
Creating and Applying (existing) domain for derivate field need_supptransofanimalsorprod
Creating and Applying (existing) domain for derivate field need_acstomechanisedequipprod
Creating and Applying (existing) domain for derivate field need_marketingsupport
Creating and Applying (existing) domain for derivate field need_cashassistance
Creating and Applying (existing) domain for derivate field need_loans
Creating and Apply